# codeclause internship

**project 2 : customer segmentation system**

**intern: vivek popat shedge**

In [1]:
import numpy as np 
import pandas as pd

In [2]:
import os

In [3]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [4]:
df_train = pd.read_csv("C:/msc-II/Train.csv")
df_test = pd.read_csv("C:/msc-II/Test.csv")
df_train.sample(4)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
5626,464478,Female,Yes,75,No,Lawyer,0.0,High,2.0,Cat_6,B
6024,465935,Male,No,27,Yes,Entertainment,9.0,Low,1.0,Cat_6,A
491,463711,Female,Yes,68,Yes,Lawyer,3.0,Low,1.0,Cat_6,A
3970,467246,Male,No,21,No,Doctor,9.0,Low,3.0,Cat_7,D


In [5]:
df_test.sample(4)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
200,459592,Male,No,25,No,Healthcare,0.0,Low,3.0,Cat_6
28,459077,Female,Yes,79,No,Lawyer,1.0,High,2.0,Cat_6
1309,463372,Female,Yes,26,No,Marketing,1.0,Average,2.0,NaN
1786,465093,Male,Yes,57,Yes,Doctor,0.0,Low,1.0,Cat_6


In [6]:
df_train.shape

(8068, 11)

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [8]:
train_null = dict(df_train.isnull().sum())
for key,val in train_null.items():
    print(key,f"======= {round((int(val)/df_train.shape[0])*100,2)} %")

ID ======= 0.0 %
Gender ======= 0.0 %
Ever_Married ======= 1.74 %
Age ======= 0.0 %
Graduated ======= 0.97 %
Profession ======= 1.54 %
Work_Experience ======= 10.28 %
Spending_Score ======= 0.0 %
Family_Size ======= 4.15 %
Var_1 ======= 0.94 %
Segmentation ======= 0.0 %


In [9]:
# here null values are very less so we can neglect them or simply drop these values
df_train.dropna(inplace=True)
df_train.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [10]:
# now removing columns 
df_train1 = df_train.drop(["ID","Var_1","Segmentation"],axis='columns')
df_train1.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size
0,Male,No,22,No,Healthcare,1.0,Low,4.0
2,Female,Yes,67,Yes,Engineer,1.0,Low,1.0
3,Male,Yes,67,Yes,Lawyer,0.0,High,2.0
5,Male,Yes,56,No,Artist,0.0,Average,2.0
6,Male,No,32,Yes,Healthcare,1.0,Low,3.0


In [11]:
# unique values for cat_cols
cat_cols = ["Gender","Ever_Married","Graduated","Profession","Spending_Score"]
for col in cat_cols:
    print(col,df_train1[col].unique())

Gender ['Male' 'Female']
Ever_Married ['No' 'Yes']
Graduated ['No' 'Yes']
Profession ['Healthcare' 'Engineer' 'Lawyer' 'Artist' 'Doctor' 'Homemaker'
 'Entertainment' 'Marketing' 'Executive']
Spending_Score ['Low' 'High' 'Average']


In [12]:
#label Encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for col in cat_cols:
    df_train1[col+"_en"] = encoder.fit_transform(df_train1[col])
df_train1.drop(cat_cols,axis='columns',inplace=True)
df_train1.head()


,Age,Work_Experience,Family_Size,Gender_en,Ever_Married_en,Graduated_en,Profession_en,Spending_Score_en
0,22,1.0,4.0,1,0,0,5,2
2,67,1.0,1.0,0,1,1,2,2
3,67,0.0,2.0,1,1,1,7,1
5,56,0.0,2.0,1,1,0,0,0
6,32,1.0,3.0,1,0,1,5,2


In [31]:
feature_cols = ['Age','Work_Experience' ,'Family_Size','Gender_en','Ever_Married_en','Graduated_en','Profession_en','Spending_Score_en']
target_col = ['Spending_Score_en']

In [32]:
from sklearn.model_selection import train_test_split
data_x = df_train1.loc[:, feature_cols]
data_y = df_train1.loc[:, target_col]
train_x,test_x,train_y,test_y = train_test_split(data_x, data_y, test_size=0.2)
print(train_x.shape)

(5332, 8)


In [33]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(5332, 8)
(5332, 1)
(1333, 8)
(1333, 1)


In [34]:
from sklearn.naive_bayes import CategoricalNB
cnb = CategoricalNB()
cnb.fit(train_x.to_numpy(), train_y.to_numpy().reshape(-1))

CategoricalNB()

In [35]:
actual_y = test_y.to_numpy().reshape(-1)
predicted_y = cnb.predict(test_x.to_numpy())

In [36]:
from sklearn import metrics
metrics.accuracy_score(actual_y, predicted_y)

1.0

In [37]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(actual_y, predicted_y))

[[310   0   0]
 [  0 212   0]
 [  0   0 811]]


In [39]:
print(classification_report(actual_y, predicted_y, target_names=[ 'B','C', 'A']))

              precision    recall  f1-score   support

           B       1.00      1.00      1.00       310
           C       1.00      1.00      1.00       212
           A       1.00      1.00      1.00       811

    accuracy                           1.00      1333
   macro avg       1.00      1.00      1.00      1333
weighted avg       1.00      1.00      1.00      1333



**thank you!**